In [1]:
import tensorflow as tf
from tensorflow import keras as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Softmax
import numpy as np

tf.__version__

'1.9.0'

In [4]:
sentences = [ "i like dog", "i love coffee", "i hate milk"]

word_list = " ".join(sentences).split()
word_list = list(set(word_list))
word_dict = {w: i for i, w in enumerate(word_list)}
number_dict = {i: w for i, w in enumerate(word_list)}
n_class = len(word_dict) # number of Vocabulary

# NNLM Parameter
n_step = 2 # number of steps ['i like', 'i love', 'i hate']
n_hidden = 2 # number of hidden units

def make_batch(sentences):
    input_batch = []
    target_batch = []

    for sen in sentences:
        word = sen.split()
        input = [word_dict[n] for n in word[:-1]]
        target = word_dict[word[-1]]

        input_batch.append(np.eye(n_class)[input])
        target_batch.append(np.eye(n_class)[target])

    return np.array(input_batch), np.array(target_batch)

# Model
model = Sequential()
model.add(Flatten()) # [batch_size, n_step * n_class]
model.add(Dense(n_hidden, activation='tanh', kernel_initializer='random_normal'))
model.add(Dense(n_class, activation='softmax'))

# loss lr=0.001 is too small
model.compile(optimizer=K.optimizers.Adam(lr=0.01),loss='categorical_crossentropy', metrics=['acc'])

# Train
input_batch, target_batch = make_batch(sentences)
his = model.fit(x=input_batch,y=target_batch, epochs=5000, verbose=0)
'Final loss:{.5} acc:{}'.format(his.history['loss'][-1], his.history['acc'][-1])

'Final loss:0.00035896702320314944 acc:1.0'

In [3]:
# Predict
predict = model.predict(input_batch)

# Test
inputs = [sen.split()[:2] for sen in sentences]
print([sen.split()[:2] for sen in sentences], '->', [number_dict[np.argmax(n)] for n in predict])

[['i', 'like'], ['i', 'love'], ['i', 'hate']] -> ['dog', 'coffee', 'milk']
